This notebook provides basic examples of how to use gp3. Use NBViewer to view the notebook [here](https://nbviewer.jupyter.org/github/as4529/gp3/blob/master/examples/basic.ipynb?flush_cache=true).

In [1]:
from gp3.inference import MFSVI, Laplace
from gp3.likelihoods import Poisson
from gp3.utils import data as sim
from gp3.utils.transforms import softplus, inv_softplus
from gp3.kernels import RBF
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from IPython.display import display
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')
from tqdm import trange
import numpy as np

## Data Simulation

First, let's simulate some data on an equispaced grid. We will do this in 2D for sake of visualization. We simulate from the following model

$$ f \sim \mathcal{GP}(\mu(\cdot), K(\cdot, \cdot))$$
$$y_i \sim \text{Poisson}(f(x_i) + \epsilon) $$

where $\epsilon \sim \mathcal{N}(0, 1)$. We can ignore the "inv_softplus" link below. It's for kernel learning (which is in progress).


In [2]:
X = sim.sim_X_equispaced(D = 2, N_dim = 20, lower=0, upper=100)
f = sim.sim_f(X, RBF(40., 1., 0.5), mu = 5.)
y = sim.poisson_draw(f, 1.) 

In [3]:
trace_func = go.Scatter3d(x = X[:,0], y = X[:,1], z=f, mode = 'markers', marker=dict(size = 2,))
trace_draws = go.Scatter3d(x = X[:,0], y = X[:,1], z=y, mode = 'markers', marker=dict(size = 2,))
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_func, 1, 1)
fig.append_trace(trace_draws, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



## Inference

Now, we run inference using both the SVI method and the Laplace method.

In [4]:
inf_svi = MFSVI(X, y, RBF(40., 1., 1.), Poisson())
inf_lp = Laplace(X, y, RBF(40., 1., 0.1), Poisson())
inf_svi.run(2000)
inf_lp.run(20)

Objective: -516801.31 | Step Size: 0.00:   0%|          | 0/20 [00:00<?, ?it/s]

('converged at', 462, 'iterations')


Here, we make predictions and plot the inferred functions.

In [5]:
pred_svi = inf_svi.predict()
pred_lp = inf_lp.f_pred

trace_svi = go.Scatter3d(x = X[:,0], y = X[:,1], z=pred_svi, mode = 'markers', marker=dict(size = 2,), name = 'SVI posterior mean')
trace_lp = go.Scatter3d(x = X[:,0], y = X[:,1], z=pred_lp, mode = 'markers', marker=dict(size = 2,), name = 'Laplace posterior mean')
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_svi, 1, 1)
fig.append_trace(trace_lp, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



To get the predictive variances, we do the following. For the SVI method, we already have them calculated. We use covariance estimation with Gaussian perturbations for the Laplace method, where the parameter indicates the number of samples to estimate based on. See "Massively Scalable GPs" for more on this method."

In [6]:
svi_variances = np.exp(inf_svi.q_S)
lp_variances = inf_lp.variance(50)

var_svi = go.Scatter3d(x = X[:,0], y = X[:,1], z=svi_variances, mode = 'markers', marker=dict(size = 2,), name = 'SVI posterior variances')
var_lp = go.Scatter3d(x = X[:,0], y = X[:,1], z=lp_variances, mode = 'markers', marker=dict(size = 2,), name = 'Laplace posterior variances')
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(var_svi, 1, 1)
fig.append_trace(var_lp, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



For SVI, we can look at the values of the variational objective, likelihood, and KL terms as a function of iteration.

In [7]:
iplot([go.Scatter(x = np.array(range(len(inf_svi.elbos)))*50, y = inf_svi.elbos)])

## Partial Grids

Here, we take a sample of 25 percent of the above grid to "observe"

In [8]:
X_part, y_part = sim.rand_partial_grid(X, y, 0.3)
X_full, y_full, obs_idx, imag_idx = sim.fill_grid(X_part, y_part)

color = np.zeros(X_full.shape[0])
color[obs_idx] = 1.0
trace_partial_obs = go.Scatter3d(x = X_full[obs_idx, 0], y = X_full[obs_idx, 1],
                                 z= y[obs_idx], mode = 'markers', marker=dict(size = 2))
iplot([trace_partial_obs])

We can run inference on partial grids by passing in the locations of the full grid, the indices of the observed points, and the values of y at the observed locations.

In [9]:
inf_svi = MFSVI(X, y_part, RBF(40., 1., 1.), Poisson(), obs_idx = obs_idx)
inf_lp = Laplace(X, y_part, RBF(40., 1., 0.1), Poisson(), obs_idx = obs_idx)
inf_svi.run(5000, n_samples = 1)
inf_lp.run(10)

Objective: -158910.42 | Step Size: 0.00:   0%|          | 0/10 [00:00<?, ?it/s]

('converged at', 1741, 'iterations')


We can make predictions of the entire function below.

In [10]:
pred_svi = inf_svi.predict()
pred_lp = inf_lp.f_pred

trace_svi = go.Scatter3d(x = X[:,0], y = X[:,1], z=pred_svi, mode = 'markers', marker=dict(size = 2, color = color), name = "SVI partial grid posterior mean")
trace_lp = go.Scatter3d(x = X[:,0], y = X[:,1], z=pred_lp, mode = 'markers', marker=dict(size = 2, color = color), name = "Laplace partial grid posterior mean")
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_svi, 1, 1)
fig.append_trace(trace_lp, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



We can get predictive variances in the same way as the full grid case.

In [11]:
svi_variances = softplus(inf_svi.q_S)
lp_variances = inf_lp.variance(50)

var_svi = go.Scatter3d(x = X[:,0], y = X[:,1], z=svi_variances, mode = 'markers', marker=dict(size = 2, color = color))
var_lp = go.Scatter3d(x = X[:,0], y = X[:,1], z=lp_variances, mode = 'markers', marker=dict(size = 2, color = color))
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(var_svi, 1, 1)
fig.append_trace(var_lp, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



In [12]:
iplot([go.Scatter(x = range(len(inf_svi.elbos)), y = inf_svi.elbos)])